In [1]:
from pyomo.environ import *

In [2]:
#data
connections = {
    (1,2):1,
    (1,4):1,
    (2,1):1,
    (2,3):1,
    (2,5):1,
    (3,2):1,
    (3,5):1,
    (3,6):1,
    (4,1):1,
    (4,5):1,
    (5,2):1,
    (5,3):1,
    (5,4):1,
    (5,6):1,
    (6,3):1,
    (6,5):1,
}   

In [10]:
#Model
m = ConcreteModel()

#define sets
m.nodes = Set(initialize = range(1, 7))
adj = list(connections.keys()) #converting keys of connections into a list
m.A = Set(within = m.nodes*m.nodes, initialize=adj) #defining adjacency matrix

#Parameters
m.connections = Param(m.A, initialize=connections)

#Decision variables
m.x = Var(m.nodes, domain=Binary)
m.y = Var(m.A, domain=Binary)

#Constraint
def max_edge_cover(m, i, j):
    return (m.x[i] + m.x[j]) >= m.y[i,j] 
m.ProjectConstraint1 = Constraint(m.A, rule=max_edge_cover)

def abc(m, i, j):
    return (m.y[i,j] >= m.x[i])
m.ProjectConstraint2 = Constraint(m.A, rule=abc)

def xyz(m, i, j):
    return (m.y[i,j] >= m.x[j])
m.ProjectConstraint3 = Constraint(m.A, rule=xyz)

def pqr(m):
    return sum(m.x[i] for i in m.nodes) <= 2
m.ProjectConstraint4 = Constraint(rule=pqr)

#Objective function
m.obj = Objective(expr = sum(m.y[i,j] for [i,j] in m.A)/2, sense=maximize)

#Solve the model
SolverFactory('gurobi').solve(m)

#print result
print("Selected nodes:")
for i in m.nodes:
    if(m.x[i].value == 1):
        print(f"Node {i}")

print(f"Total number of edges covered: {m.obj.expr()}")

Selected nodes:
Node 3
Node 5
Total number of edges covered: 6.0
